In [ ]:
# GENERAL SCRIPT FOR ADVERSARIAL TRAINING -- COPY AND MODIFY FOR TRAINING  
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

from torch.autograd import Variable
import numpy as np


# Universal import block 
# Block to get the relative imports working 
import os
import sys 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import config
import matplotlib.pyplot as plt 
import prebuilt_loss_functions as plf
import loss_functions as lf 
import utils.pytorch_utils as utils
import utils.image_utils as img_utils
import cifar10.cifar_loader as cifar_loader
import cifar10.cifar_resnets as cifar_resnets
import adversarial_attacks as aa
import adversarial_training as advtrain
import adversarial_evaluation as adveval
import utils.checkpoints as checkpoints
import adversarial_perturbations as ap
import adversarial_attacks_refactor as aar 
import spatial_transformers as st 
reload(ap)

In [ ]:
# GLOBAL VARS 
DATASET = 'CIFAR10' # alternative is 'IMAGENET'
USE_GPU = torch.cuda.is_available()

TRAIN_BATCH_SIZE = 128 
VAL_BATCH_SIZE = 16 

CIFAR_ARCHITECTURE = 'resnet32' # If you change, change in code too
IMAGENET_ARCHITECTURE = 'nasnetalarge' 
EXPERIMENT_NAME = 'reg_adv_test_PGD_Linf8_Lr01_Iter50'


In [ ]:
""" Goal here is to make sure adversarial training works under the refactored model. It _should_, but it might need 
    a few tweaks 
"""

if DATASET == 'CIFAR10':
    classifier_net, normalizer = cifar_loader.load_pretrained_cifar_resnet(flavor=32,
                                                                           use_gpu=USE_GPU,
                                                                           return_normalizer=True)
    classifier_net.eval()
    train_loader = cifar_loader.load_cifar_data('train', normalize=False, 
                                                batch_size=TRAIN_BATCH_SIZE, 
                                                use_gpu=USE_GPU)
    val_loader = cifar_loader.load_cifar_data('val', normalize=False, 
                                              batch_size=VAL_BATCH_SIZE, 
                                              use_gpu=USE_GPU)
    ARCHITECTURE = CIFAR_ARCHITECTURE
    
else:
    classifier_net, normalizer = imagenet_loader.load_pretrained_imagenet(arch=IMAGENET_ARCHITECTURE,
                                                                          use_gpu=USE_GPU, 
                                                                          return_normalizer=True)
    classifier_net.eval() 
    train_loader = imagenet_loader.load_imagenet_data('train', normalize=False, 
                                                      batch_size=TRAIN_BATCH_SIZE, 
                                                      use_gpu=USE_GPU)
    val_loader = imagenet_loader.load_imagenet_data('val', normalize=False, 
                                                    batch_size=VAL_BATCH_SIZE, 
                                                    use_gpu=USE_GPU)
    ARCHITECTURE = IMAGENET_ARCHITECTURE

examples, labels = next(iter(val_loader))
if USE_GPU:
    examples = examples.cuda()
    labels = labels.cuda() 
    

In [ ]:
# BUILD ATTACK PARAMETERS HERE 
L_INF_BOUND = 8.0 / 255.0
def build_pgd_linf_attack_params():
    # Threat model first 

    delta_threat = ap.ThreatModel(ap.DeltaAddition, 
                                  ap.PerturbationParameters(lp_style='inf',
                                                            lp_bound=L_INF_BOUND, 
                                                            use_gpu=USE_GPU))
    attack_loss = plf.VanillaXentropy(classifier_net, normalizer=normalizer)
    pgd_attack = aar.PGD(classifier_net, normalizer, delta_threat, attack_loss)
    pgd_kwargs = {'num_iterations': 50,
                  'optimizer': optim.Adam, 
                  'optimizer_kwargs': {'lr': 0.01},
                  'signed': False, 
                  'verbose': False}
    params = advtrain.AdversarialAttackParameters(pgd_attack, 1.0, 
                                                  attack_specific_params={'attack_kwargs': pgd_kwargs})
    return params 


pgd_linf_attack_params = build_pgd_linf_attack_params()

In [ ]:
classifier_net.train()
train_obj = advtrain.AdversarialTraining(classifier_net, normalizer, 
                                         EXPERIMENT_NAME, ARCHITECTURE)

train_loss = nn.CrossEntropyLoss() # USE A PREBUILT TRAIN LOSS FXN 
train_obj.train(train_loader, 2, train_loss, 
                attack_parameters=pgd_linf_attack_params, 
                verbosity="snoop", 
                regularize_adv_scale=100.0,
                use_gpu=USE_GPU)

